In [93]:
import kagglehub
import os
import pandas as pd
import requests
import re
import nltk
import spacy 


In [94]:
# Download dataset latest version
path = kagglehub.dataset_download("mohamedbakhet/amazon-books-reviews")

print("Path to dataset files:", path)

# Listar os ficheiros no diretório descarregado
files = os.listdir(path)
print("Ficheiros disponíveis:", files)


Path to dataset files: C:\Users\Margarida\.cache\kagglehub\datasets\mohamedbakhet\amazon-books-reviews\versions\1
Ficheiros disponíveis: ['books_data.csv', 'Books_rating.csv']


In [95]:
# Caminhos completos para os ficheiros CSV
file1 = os.path.join(path, "books_data.csv") 
file2 = os.path.join(path, "Books_rating.csv")

# Carregar os CSVs em DataFrames
books_data = pd.read_csv(file1)
books_rating = pd.read_csv(file2)

# Verificar os primeiros registos
print(books_data.head())
print(books_rating.head())

                                               Title  \
0                     Its Only Art If Its Well Hung!   
1                           Dr. Seuss: American Icon   
2              Wonderful Worship in Smaller Churches   
3                      Whispers of the Wicked Saints   
4  Nation Dance: Religion, Identity and Cultural ...   

                                         description              authors  \
0                                                NaN     ['Julie Strain']   
1  Philip Nel takes a fascinating look into the k...       ['Philip Nel']   
2  This resource includes twelve principles in un...     ['David R. Ray']   
3  Julia Thomas finds her life spinning out of co...  ['Veronica Haddon']   
4                                                NaN      ['Edward Long']   

                                               image  \
0  http://books.google.com/books/content?id=DykPA...   
1  http://books.google.com/books/content?id=IjvHQ...   
2  http://books.google.com/books

In [96]:
books_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Id                  object 
 1   Title               object 
 2   Price               float64
 3   User_id             object 
 4   profileName         object 
 5   review/helpfulness  object 
 6   review/score        float64
 7   review/time         int64  
 8   review/summary      object 
 9   review/text         object 
dtypes: float64(2), int64(1), object(7)
memory usage: 228.9+ MB


In [97]:
books_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212404 entries, 0 to 212403
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Title          212403 non-null  object 
 1   description    143962 non-null  object 
 2   authors        180991 non-null  object 
 3   image          160329 non-null  object 
 4   previewLink    188568 non-null  object 
 5   publisher      136518 non-null  object 
 6   publishedDate  187099 non-null  object 
 7   infoLink       188568 non-null  object 
 8   categories     171205 non-null  object 
 9   ratingsCount   49752 non-null   float64
dtypes: float64(1), object(9)
memory usage: 16.2+ MB


In [98]:
NaN_books_data = books_data.isnull().sum()
print(NaN_books_data)

Title                 1
description       68442
authors           31413
image             52075
previewLink       23836
publisher         75886
publishedDate     25305
infoLink          23836
categories        41199
ratingsCount     162652
dtype: int64


In [99]:
books_data_semNaN = books_data.dropna(subset=['Title', 'authors', 'categories', 'description'])

In [100]:
books_data_semNaN.info()

<class 'pandas.core.frame.DataFrame'>
Index: 136138 entries, 1 to 212402
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Title          136138 non-null  object 
 1   description    136138 non-null  object 
 2   authors        136138 non-null  object 
 3   image          130818 non-null  object 
 4   previewLink    136138 non-null  object 
 5   publisher      121139 non-null  object 
 6   publishedDate  135823 non-null  object 
 7   infoLink       136138 non-null  object 
 8   categories     136138 non-null  object 
 9   ratingsCount   43884 non-null   float64
dtypes: float64(1), object(9)
memory usage: 11.4+ MB


In [101]:
books_data_semNaN.drop(columns=['previewLink', 'publisher', 'publishedDate', 'infoLink'], inplace=True)

C:\Users\Margarida\AppData\Local\Temp\ipykernel_17608\874535685.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books_data_semNaN.drop(columns=['previewLink', 'publisher', 'publishedDate', 'infoLink'], inplace=True)


In [102]:
books_data_semNaN.info()

<class 'pandas.core.frame.DataFrame'>
Index: 136138 entries, 1 to 212402
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Title         136138 non-null  object 
 1   description   136138 non-null  object 
 2   authors       136138 non-null  object 
 3   image         130818 non-null  object 
 4   categories    136138 non-null  object 
 5   ratingsCount  43884 non-null   float64
dtypes: float64(1), object(5)
memory usage: 7.3+ MB


# redução da amostra, porque não estavamos a conseguir executar o código

In [103]:
# Realizar a amostragem estratificada manualmente
sample_booksdata = books_data_semNaN.groupby('categories', group_keys=False).apply(lambda x: x.sample(frac=0.5, random_state=42))

print(sample_booksdata)

C:\Users\Margarida\AppData\Local\Temp\ipykernel_17608\2162787207.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_booksdata = books_data_semNaN.groupby('categories', group_keys=False).apply(lambda x: x.sample(frac=0.5, random_state=42))


                                                    Title  \
13005                             What's Wrong With Daddy   
19051   The 36-Hour Day: A Family Guide to Caring for ...   
147229  Twilight Travels with Mother: How I Found Stre...   
203597                                          Pure Pulp   
176575  The Man Who Lost America: a Biography 0f Gentl...   
...                                                   ...   
145828           Crazy for You (Sweet Dreams Series #145)   
186235                        Jake Finds Out (Making Out)   
51608                                     Born of the Sun   
211224      The Whispering Knights: Complete & Unabridged   
169094  The Zen teaching of Huang Po on the transmissi...   

                                              description  \
13005   "Jennifer Burke sat between her mother and fat...   
19051   Provides practical and legal advice on caring ...   
147229  For any person impacted by Alzheimer's disease...   
203597  A survey of wor

In [104]:
# redução da amostra das reviews, com base na sample dos livros
sample_booksrating = books_rating[books_rating['Title'].isin(sample_booksdata['Title'])]

In [105]:
sample_booksrating.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1086251 entries, 1 to 2999999
Data columns (total 10 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   Id                  1086251 non-null  object 
 1   Title               1086251 non-null  object 
 2   Price               185495 non-null   float64
 3   User_id             882886 non-null   object 
 4   profileName         882840 non-null   object 
 5   review/helpfulness  1086251 non-null  object 
 6   review/score        1086251 non-null  float64
 7   review/time         1086251 non-null  int64  
 8   review/summary      1086111 non-null  object 
 9   review/text         1086247 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 91.2+ MB


# pré-processamento da sample de book's data

In [106]:
sample_booksdata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 66476 entries, 13005 to 169094
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         66476 non-null  object 
 1   description   66476 non-null  object 
 2   authors       66476 non-null  object 
 3   image         64238 non-null  object 
 4   categories    66476 non-null  object 
 5   ratingsCount  21550 non-null  float64
dtypes: float64(1), object(5)
memory usage: 3.6+ MB


In [107]:
#lowercasing
sample_booksdata = sample_booksdata.map(lambda s: s.lower() if type(s) == str else s)

In [108]:
#ponctuation removal
sample_booksdata = sample_booksdata.map(lambda x: re.sub(r'[^\w\s]', '', x) if isinstance(x, str) else x)  # Remove special characters
sample_booksdata = sample_booksdata.map(lambda x: re.sub(r'\d+', '', x) if isinstance(x, str) else x)  # Remove numbers
sample_booksdata = sample_booksdata.map(lambda x: re.sub(r'º', '', x) if isinstance(x, str) else x)  # Remove the "º" character
sample_booksdata = sample_booksdata.map(lambda x: ' '.join(x.split()) if isinstance(x, str) else x)  # Remove extra spaces

In [109]:
# remove stopwords
from nltk.corpus import stopwords

nltk.download('stopwords')

# Definir a lista de stopwords
stop_words = set(stopwords.words('english'))

# Função para remover stopwords
def remove_stopwords(text):
    words = text.split()
    cleaned_text = ' '.join([word for word in words if word.lower() not in stop_words])
    return cleaned_text

# Converter os valores das colunas para string antes de aplicar a função
sample_booksdata['description'] = sample_booksdata['description'].astype(str).apply(remove_stopwords)

print(sample_booksdata)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Margarida\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                                    Title  \
13005                              whats wrong with daddy   
19051   the hour day a family guide to caring for pers...   
147229  twilight travels with mother how i found stren...   
203597                                          pure pulp   
176575  the man who lost america a biography f gentlem...   
...                                                   ...   
145828                  crazy for you sweet dreams series   
186235                          jake finds out making out   
51608                                     born of the sun   
211224         the whispering knights complete unabridged   
169094  the zen teaching of huang po on the transmissi...   

                                              description  \
13005   jennifer burke sat mother father doctors offic...   
19051   provides practical legal advice caring longer ...   
147229  person impacted alzheimers disease mayos work ...   
203597  survey works le

In [110]:
# Remove words that are shorter than or equal to 2 characters
# Função para remover palavras curtas
def remove_short_words(text):
    return ' '.join([word for word in text.split() if len(word) > 2])

# Aplicar a função de remoção de palavras curtas à coluna
sample_booksdata['description'] = sample_booksdata['description'].apply(remove_short_words)

print(sample_booksdata)

                                                    Title  \
13005                              whats wrong with daddy   
19051   the hour day a family guide to caring for pers...   
147229  twilight travels with mother how i found stren...   
203597                                          pure pulp   
176575  the man who lost america a biography f gentlem...   
...                                                   ...   
145828                  crazy for you sweet dreams series   
186235                          jake finds out making out   
51608                                     born of the sun   
211224         the whispering knights complete unabridged   
169094  the zen teaching of huang po on the transmissi...   

                                              description  \
13005   jennifer burke sat mother father doctors offic...   
19051   provides practical legal advice caring longer ...   
147229  person impacted alzheimers disease mayos work ...   
203597  survey works le

In [126]:
#Devolver a coluna original de authors à sample_booksdata
sample_booksdata['authors'] = books_data['authors']

print(sample_booksdata['authors'])

13005                        ['Alida E. Young']
19051      ['Nancy L. Mace', 'Peter V. Rabins']
147229                        ['Mary Ann Mayo']
203597                       ['Bridget Donlon']
176575    ['Noel Bertram Gerson', 'Paul Lewis']
                          ...                  
145828                       ['Jahnna Beecham']
186235                  ['Katherine Applegate']
51608                         ['Gillian Cross']
211224                      ['Penelope Lively']
169094            ['黄檗', 'Po Huang', 'Xiu Pei']
Name: authors, Length: 66476, dtype: object


In [135]:
sample_booksdata.drop(columns=['ratingsCount'], inplace=True)
sample_booksdata.head()

,Title,description,authors,image,categories,title_raw
13005,What's Wrong With Daddy,jennifer burke sat mother father doctors offic...,['Alida E. Young'],httpbooksgooglecombookscontentidhwyfzspzakgcpr...,alzheimers disease,What's Wrong With Daddy
19051,The 36-Hour Day: A Family Guide to Caring for ...,provides practical legal advice caring longer ...,"['Nancy L. Mace', 'Peter V. Rabins']",NaN,alzheimers disease,The 36-Hour Day: A Family Guide to Caring for ...
147229,Twilight Travels with Mother: How I Found Stre...,person impacted alzheimers disease mayos work ...,['Mary Ann Mayo'],NaN,alzheimers disease,Twilight Travels with Mother: How I Found Stre...
203597,Pure Pulp,survey works leading contemporary artists made...,['Bridget Donlon'],httpbooksgooglecombookscontentidrjkdaeacaajpri...,artists books,Pure Pulp
176575,The Man Who Lost America: a Biography 0f Gentl...,general john burgoyne february august british ...,"['Noel Bertram Gerson', 'Paul Lewis']",httpbooksgooglecombookscontentidpdbaaaaiaajpri...,burgoynes invasion,The Man Who Lost America: a Biography 0f Gentl...


In [136]:
#Devolver a coluna original de titulo à sample_booksdata
sample_booksdata['title_raw'] = books_data['Title']

print(sample_booksdata['title_raw'])

13005                               What's Wrong With Daddy
19051     The 36-Hour Day: A Family Guide to Caring for ...
147229    Twilight Travels with Mother: How I Found Stre...
203597                                            Pure Pulp
176575    The Man Who Lost America: a Biography 0f Gentl...
                                ...                        
145828             Crazy for You (Sweet Dreams Series #145)
186235                          Jake Finds Out (Making Out)
51608                                       Born of the Sun
211224        The Whispering Knights: Complete & Unabridged
169094    The Zen teaching of Huang Po on the transmissi...
Name: title_raw, Length: 66476, dtype: object


# pré-processamento da sample de book's ratings

In [113]:
sample_booksrating.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1086251 entries, 1 to 2999999
Data columns (total 10 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   Id                  1086251 non-null  object 
 1   Title               1086251 non-null  object 
 2   Price               185495 non-null   float64
 3   User_id             882886 non-null   object 
 4   profileName         882840 non-null   object 
 5   review/helpfulness  1086251 non-null  object 
 6   review/score        1086251 non-null  float64
 7   review/time         1086251 non-null  int64  
 8   review/summary      1086111 non-null  object 
 9   review/text         1086247 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 91.2+ MB


In [114]:
NaN_books_rating = sample_booksrating.isnull().sum()
print(NaN_books_rating)

Id                         0
Title                      0
Price                 900756
User_id               203365
profileName           203411
review/helpfulness         0
review/score               0
review/time                0
review/summary           140
review/text                4
dtype: int64


In [115]:
# dropar as linhas com NaN
sample_booksrating = sample_booksrating.dropna(subset=['review/summary', 'review/text'])

In [137]:
#dropar colunas desnecessárias
sample_booksrating.drop(columns=['Price', 'review/helpfulness'], inplace=True)

In [138]:
sample_booksrating.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1086107 entries, 1 to 2999999
Data columns (total 8 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   Id              1086107 non-null  object 
 1   Title           1086107 non-null  object 
 2   User_id         882748 non-null   object 
 3   profileName     882704 non-null   object 
 4   review/score    1086107 non-null  float64
 5   review/time     1086107 non-null  int64  
 6   review/summary  1086107 non-null  object 
 7   review/text     1086107 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 74.6+ MB


In [117]:
#lowercasing
sample_booksrating = sample_booksrating.map(lambda s: s.lower() if type(s) == str else s)

In [118]:
#ponctuation removal
sample_booksrating = sample_booksrating.map(lambda x: re.sub(r'[^\w\s]', '', x) if isinstance(x, str) else x)  # Remove special characters
sample_booksrating = sample_booksrating.map(lambda x: re.sub(r'\d+', '', x) if isinstance(x, str) else x)  # Remove numbers
sample_booksrating = sample_booksrating.map(lambda x: re.sub(r'º', '', x) if isinstance(x, str) else x)  # Remove the "º" character
sample_booksrating = sample_booksrating.map(lambda x: ' '.join(x.split()) if isinstance(x, str) else x)  # Remove extra spaces

In [139]:
# remove stopwords
from nltk.corpus import stopwords

nltk.download('stopwords')

# Definir a lista de stopwords
stop_words = set(stopwords.words('english'))

# Função para remover stopwords
def remove_stopwords(text):
    words = text.split()
    cleaned_text = ' '.join([word for word in words if word.lower() not in stop_words])
    return cleaned_text

# Converter os valores das colunas para string antes de aplicar a função
sample_booksrating['review/summary'] = sample_booksrating['review/summary'].astype(str).apply(remove_stopwords)
sample_booksrating['review/text'] = sample_booksrating['review/text'].astype(str).apply(remove_stopwords)

print(sample_booksrating)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Margarida\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


              Id                   Title      User_id  \
1                 dr seuss american icon     atkudnsr   
2                 dr seuss american icon    auhuzrsvo   
3                 dr seuss american icon     amvuwtqh   
4                 dr seuss american icon    axxupkfmr   
5                 dr seuss american icon  afnonfudbuk   
...          ...                     ...          ...   
2999995  bnslvcu     the idea of history          NaN   
2999996  bnslvcu     the idea of history   asmubaslly   
2999997  bnslvcu     the idea of history  aaqmekzkkee   
2999998  bnslvcu     the idea of history    asqgybksk   
2999999  bnslvcu     the idea of history          NaN   

                             profileName  review/score  review/time  \
1                          kevin killian           5.0   1095724800   
2                           john granger           5.0   1078790400   
3        roy e perry amateur philosopher           4.0   1090713600   
4            d h richards ninth

In [120]:
# Remove words that are shorter than or equal to 2 characters
# Função para remover palavras curtas
def remove_short_words(text):
	return ' '.join([word for word in text.split() if len(word) > 2])

# Aplicar a função de remoção de palavras curtas às colunas
sample_booksrating['review/summary'] = sample_booksrating['review/summary'].apply(remove_short_words)
sample_booksrating['review/text'] = sample_booksrating['review/text'].apply(remove_short_words)

print(sample_booksrating.head())

  Id                   Title  Price      User_id  \
1     dr seuss american icon    NaN     atkudnsr   
2     dr seuss american icon    NaN    auhuzrsvo   
3     dr seuss american icon    NaN     amvuwtqh   
4     dr seuss american icon    NaN    axxupkfmr   
5     dr seuss american icon    NaN  afnonfudbuk   

                       profileName review/helpfulness  review/score  \
1                    kevin killian                              5.0   
2                     john granger                              5.0   
3  roy e perry amateur philosopher                              4.0   
4      d h richards ninthwavestore                              4.0   
5                           malvin                              4.0   

   review/time                                 review/summary  \
1   1095724800                                 really enjoyed   
2   1078790400        essential every personal public library   
3   1090713600  phlip nel gives silly seuss serious treatment   


# salvar amostras num csv

In [140]:

#verificar diretório de trabalho
print(os.getcwd())

c:\PAC


In [141]:
sample_booksdata.to_csv('sample_booksdata.csv', index=False)
sample_booksrating.to_csv('sample_booksrating.csv', index=False)